**Proyecto:** INE Incidentes de Transito.

**Objetivo:** Consultas con los principales resultados.

**Dependencias / Librerias:** notebook transform

**Ambiente de Ejecución:** Databricks, Desarrollo

---
| Fecha | Desarrollador | Descripcion |
| :--- | :--- | :--- |
| 10/NOV/2025 | Ruben Coloma | Emision Inicial |

In [0]:
from pyspark.sql import functions as F

In [0]:
}

In [0]:
# Cargar las tablas del catálogo
hechos = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.th_fallecidos_lesionados")
dim_depto = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_depto_ocu")
dim_mes = spark.table("cat_ine_trafico_dev.silver_ine_trafico_dev.dim_ine_mes_ocu")

# Unir las tablas
df = (
    hechos
    .join(dim_depto, "id_depto_ocu", "left")
    .join(dim_mes, "id_mes_ocurrio", "left")
)

# Agrupar por departamento y mes, contando incidentes
df_agregado = (
    df.groupBy("depto_ocu", "mes_mezcla")
      .agg(F.count("*").alias("total_incidentes"))
)

# Pivotear: columnas = meses, filas = departamentos
df_pivot = (
    df_agregado.groupBy("depto_ocu")
               .pivot("mes_mezcla")
               .agg(F.sum("total_incidentes"))
)

# Agregar columna TOTAL (suma de todos los meses)
# Se suman dinámicamente todas las columnas excepto depto_ocu
cols_mes = [c for c in df_pivot.columns if c not in ["depto_ocu"]]
df_final = df_pivot.withColumn("TOTAL", sum(F.coalesce(F.col(c), F.lit(0)) for c in cols_mes))

# Renombrar columna depto_ocu a MES (como pediste)
df_final = df_final.withColumnRenamed("depto_ocu", "MES")

# Mostrar el resultado
display(df_final.orderBy("MES"))

In [0]:
%sql
SELECT  